In [1]:
import pickle
import re
from tqdm import tqdm
import pandas as pd
import numpy as np
import csv

In [2]:
import findspark
findspark.init('/home/lss9/anaconda3/lib/python3.7/site-packages/pyspark/')
from pyspark.sql import SQLContext
from pyspark import SparkContext, SparkConf
import re
from tqdm import tqdm

In [3]:
conf = SparkConf().setAll([('spark.executor.memory', '16g'), ('spark.executor.cores', '3'), ('spark.cores.max', '3'), ('spark.driver.memory','16g')])
sc = SparkContext(conf=conf)
sqlContext = SQLContext(sc)

In [4]:
sc.getConf().getAll()

[('spark.driver.port', '35545'),
 ('spark.driver.host', 'gpu02.cin.ufpe.br'),
 ('spark.executor.id', 'driver'),
 ('spark.driver.memory', '16g'),
 ('spark.app.name', 'pyspark-shell'),
 ('spark.cores.max', '3'),
 ('spark.executor.memory', '16g'),
 ('spark.rdd.compress', 'True'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.master', 'local[*]'),
 ('spark.executor.cores', '3'),
 ('spark.submit.deployMode', 'client'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.app.id', 'local-1566305676530')]

In [5]:
def clearString(text):
    
    text = re.sub('[^A-Za-z]+',' ',text)
    
    text = text.lstrip()
    
    text = text.rstrip()
    
    return text

In [6]:
rawTables = pickle.load(open("articlesDataset/final_dict_tables.pkl","rb"))

In [7]:
rawData = sqlContext.read.json('/home/luciano/google/tables.json')

In [8]:
rawData.registerTempTable("tables")

In [9]:
#getting distinct keys
Keys = []

for i in tqdm(rawTables.keys()):

    dataTable = rawTables[i]
    
    totalTalbes = len(dataTable)
    
    for j in range(totalTalbes):
        
        tableKey = dataTable[j][0][0]
            
        Keys.append(tableKey)

100%|██████████| 90000/90000 [00:00<00:00, 603492.15it/s]


In [10]:
aux_list = set(Keys)
distinctKeys = list(aux_list)

In [11]:
formattedTables = []

first = 0
last = 4999

#first = 50000
#last = 54999

for i in range(18):
    
    auxString = "select pgId, pgTitle, sectionTitle, tableCaption, tableHeaders, tableData from tables where pgId in ("

    auxString = auxString +''+ str(distinctKeys[first])

    for k in range(first,last):
    
        auxString = auxString +','+ str(distinctKeys[k])

    auxString = auxString +')'
    
    first = last + 1
    last = last + 5000

    #getting the query
    query = auxString

    returnSelect = sqlContext.sql(query).collect()

    for row in tqdm(returnSelect):
    
        tablePgID = row.pgId
        tablePgTitle = row.pgTitle
        tableSectionTitle = row.sectionTitle
        tableCaption = str(row.tableCaption)
    
        #removing special characters
        tablePgTitle = clearString(tablePgTitle)
        tableSectionTitle = clearString(tableSectionTitle)
        tableCaption = clearString(tableCaption)
    
        #getting the headers
        tableHeader = ''
    
        tablesHeaderRawData = row.tableHeaders
    
        for tableHeaderLines in tablesHeaderRawData:
            
            tableHeaderRawColumns = tableHeaderLines
        
            for tableHeaderColumns in tableHeaderRawColumns:
            
                tableHeader = tableHeader +' '+ tableHeaderColumns.text
            
        tableHeader = clearString(tableHeader)
    
    
        #getting the table body
        tableBody = ''
    
        tableRawData = row.tableData
    
        for tableLines in tableRawData:
            
            tableRawColumns = tableLines
        
            for tableColumns in tableRawColumns:
            
                tableBody = tableBody +' '+ tableColumns.text
            
        tableBody = clearString(tableBody)
        
        cachAllText = tablePgTitle +" "+tableSectionTitle+" "+tableCaption+" "+tableHeader+" "+tableBody
    
        # save the table in list
        formattedTables.append([tablePgID,tablePgTitle,tableSectionTitle,tableCaption,tableHeader,tableBody,cachAllText])

100%|██████████| 17003/17003 [00:03<00:00, 4971.14it/s]


In [12]:
# formattedTables = []

# for i in tqdm(rawTables.keys()):

#     dataTable = rawTables[i]
    
#     totalTalbes = len(dataTable)
    
#     for j in range(totalTalbes):
        
#         tableKey = dataTable[j][0][0]
        
#         tableCaption = dataTable[j][0][1]
        
#         tableHeader = str(dataTable[j][0][2:])
#         tableHeader = re.sub('[^A-Za-z]+',' ',tableHeader)
#         tableHeader = tableHeader.lstrip()
#         tableHeader = tableHeader.rstrip()
                
#         tableBody = str(dataTable[j][1:])
#         tableBody = re.sub('[^A-Za-z]+',' ',tableBody)
#         tableBody = tableBody.lstrip()
#         tableBody = tableBody.rstrip()
        
#         cachAllText = tableCaption +" "+tableHeader+" "+tableBody
        
#         formattedTables.append([tableKey,tableCaption,tableHeader,tableBody,cachAllText])

In [13]:
for tables in tqdm(formattedTables):
    if (tables[5] == ''):
        formattedTables.remove(tables)

 97%|█████████▋| 298793/308686 [02:32<00:05, 1961.36it/s] 


In [14]:
with open('cleanDataTablesFile', 'w') as myfile:
        
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    
    for tables in tqdm(formattedTables):
    
        tablePgID = tables[0]
    
        tablePgTitle = tables[1]
        
        tableSectionTitle = tables[2]
        
        tableCaption = tables[3]
        
        tableHeader = tables[4]
        
        tableBody = tables[5]
        
        cachAllText = tables[6]
    
        row = [tablePgID,tablePgTitle,tableSectionTitle,tableCaption,tableHeader,tableBody,cachAllText]
        
        wr.writerow(row)

100%|██████████| 298793/298793 [00:09<00:00, 30751.59it/s]
